In [2]:
#Импорт библиотек
import requests
import urllib3
urllib3.disable_warnings()
from datetime import date
from py2neo import Graph
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
#Авторизации
logopass=pd.read_excel('lp.xlsx')
neologin=logopass.query('system=="neo"').reset_index().login[0]
neopass=logopass.query('system=="neo"').reset_index().password[0]
#Авторизация neo4j
graph = Graph("https://neo4j-kanban.biocad.ru:443", auth=(neologin, neopass))
#Авторизация REPKA
login_token = logopass.query('system=="repka"').reset_index().token[0]
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
#Устанавливаем стили для графика
plt.style.use('bmh')
plt.rcParams['figure.figsize'] = 15, 6

In [ ]:
3141225 2019-04-04
3141227 2019-02-25
3141221 2024-03-29
3141377 2023-02-28

In [8]:
request="""
match (t:TaskNode) where t.bkb_id=3141377
return distinct t.name as Name, t.bkb_id as ID
"""
df=graph.run(request).to_data_frame()

replaces = {}
replaces['forecast'] = '2023-02-28'

#Дергаем API   
for i in tqdm(range(df.shape[0])):
    r = requests.patch('https://repka.biocad.ru/api/v1/tasks/'+str(int(df['ID'][i]))+'/' ,
                       json=replaces, headers = {'Authorization': 'Token ' + login_token}, verify=False)
r.status_code

200

In [4]:
df.head()

,Name,ID
0,BCD-132 [CD20] Получить разрешение на КИ 2 фа...,3141225


In [1]:
replaces = {}
replaces['forecast'] = '2021-05-20'

#Дергаем API   
for i in tqdm(range(df.shape[0])):
    r = requests.patch('https://repka.biocad.ru/api/v1/tasks/'+str(int(df['ID'][i]))+'/' ,
                       json=replaces, headers = {'Authorization': 'Token ' + login_token}, verify=False)
r.status_code

NameError: name 'tqdm' is not defined

In [10]:
#Чтение эксельки - ковидный штаб
cov=pd.read_excel("Задачи covid.xlsx")
cov.head()

,Name,Team,Team_id,Template,Template_id
0,Использование коммерческих структур в рамках О...,Covid.Управление,4738,Управление партнерствами,27860
1,"Телемедицина, план по развитию",Covid.Управление,4738,Управление партнерствами,27860
2,Диспансеризация детей ?,Covid.Управление,4738,Управление партнерствами,27860
3,План по ограничительным мерам в связке со стат...,Covid.Управление,4738,Управление партнерствами,27860
4,Проверить поликлинические очереди,Covid.Управление,4738,Управление партнерствами,27860


In [11]:
#Создание задач - ковидный штаб
replaces = {}
replaces['status'] = 'NEW'
replaces['priority'] = '200'
replaces['client']='5496'
replaces['project']='4496'
#Дергаем API   
for i in tqdm(range(cov.shape[0])):
    replaces["assigned_to_team"] = str(int(cov['Team_id'][i]))
    replaces["name"] = str(cov['Name'][i])
    replaces["task_template"] = str(int(cov['Template_id'][i]))
    r = requests.post('https://repka.biocad.ru/api/v1/tasks/' ,
                       json=replaces, headers = {'Authorization': 'Token ' + login_token}, verify=False)
r.status_code

201

In [14]:
#Задачи штаб
request="""
match (t:TaskNode)--(p:ProjectNode {bkb_id:4496})
return t.name as Name, t.bkb_id as ID
"""
df=graph.run(request).to_data_frame()

In [16]:
#Результаты под задачи штаб
replaces = {}
replaces['name'] = 'Задача выполнена'
replaces['type'] = 'OTHER'

#Дергаем API   
for i in tqdm(range(df.shape[0])):
    replaces["task"] = str(int(df['ID'][i]))
    r = requests.post('https://repka.biocad.ru/api/v1/task-additional-fields/' ,
                       json=replaces, headers = {'Authorization': 'Token ' + login_token}, verify=False)
r.status_code

KeyError: 100

In [31]:
df.shape

(102, 3)

In [32]:
#Удаление типовых задач
#replaces = {}
#replaces['is_deleted'] = 'true'
#Дергаем API   
for i in tqdm(range(df.shape[0])):
    r = requests.delete('https://repka.biocad.ru/api/v1/teams/' + str(int(df['ID'][i])) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token}, verify=False)
r.status_code

204

In [108]:
#Смена статуса
replaces = {}

#replaces['status'] = 'PLAN'
#replaces['status'] = 'IN_PROGRESS'
#Дергаем API   
for task in tqdm(range(df.shape[0])):
    replaces['status'] = 'PLAN'
    r = requests.patch('https://repka.biocad.ru/api/v1/tasks/' + str(int(df['task'][task])) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token})
    replaces['status'] = 'IN_PROGRESS'
    replaces['substatus_name'] = df['Substatus'][task]
    r = requests.patch('https://repka.biocad.ru/api/v1/tasks/' + str(int(df['task'][task])) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token})
r.status_code




200

In [81]:
r.text

'{"id":2365630,"assigned_to_username":null,"task_template":11831,"ext_fields":{"assigned_to__name":"Жоров Даниил Сергеевич","assigned_to__avatar":"https://auth.newportal.biocad.ru//uploads/user/avatar/4543/small_c42e1973-7c5f-4b31-a608-489a25ae93b7.jpg","assigned_to__username":"zhorov","assigned_to__teams":null,"assignee_avatar":"https://auth.newportal.biocad.ru//uploads/user/avatar/4543/small_c42e1973-7c5f-4b31-a608-489a25ae93b7.jpg","assignee_username":"zhorov","assigned_to_team__name":"Департамент Организационной Трансформации (ДОТ)","assigned_to_team__icon_config":{"text":"ДОТ","text_color":"#FFFFFF","stroke_color":"#AD3400","background_color":"#202226"},"assigned_to_team__board__id":10475,"project__name":null,"project__status":null,"project__has_high_priority":null,"project__code":null,"project__program":null,"project__program__name":null,"project__program__description":null,"project__pp_project_url":null,"project__pp_program_url":null,"task_template__name":"Я не могу выбрать типо

In [3]:
#Удаление мертвых команд
request="""match (team:TeamNode {is_deleted:false}) where not (team)--(:UserNode {state:"Работа"}) and not team.bkb_id in [3479,4320]
optional match (team)--(u:UserNode)
Return distinct team.name, team.bkb_id as ID, collect (u.username), collect (u.state)"""
df=graph.run(request).to_data_frame()

replaces = {}

replaces['is_deleted'] = 'true'

#Дергаем API   
for i in tqdm(range(df.shape[0])):
    r = requests.delete('https://repka.biocad.ru/api/v1/teams/' + str(int(df['ID'][i])) + '/',
                       #json=replaces, 
                        headers = {'Authorization': 'Token ' + login_token})
r.status_code

204

#Привязка типовых задач  
for i in tqdm(range(df.shape[0])):
    replaces= {}
    replaces['team']=str(int(df['id'][i]))
    replaces['task_template']='15177'
    r = requests.post('https://repka.biocad.ru/api/v1/team-task-template-links/' + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token})
    print(r.status_code)


In [53]:
r.text

'{"detail":"Не найдено.","exception":"Http404"}'

#TaskTemplatesEditing
request="""
match (tt:TaskTemplateNode)--(team:TeamNode)
where team.name contains 'Evolution' and not tt.is_deleted=true
optional match (tt)--(t:TeamNode)
return distinct tt.bkb_id as bkb, tt.name, tt.is_deleted, count (distinct t) as Teams order by Teams desc
"""
df=graph.run(request).to_data_frame()
replaces = {}

replaces['is_deleted'] = 'true'

#Дергаем API   
for i in tqdm(range(df.shape[0])):
    r = requests.patch('https://repka.biocad.ru/api/v1/task-templates/' + str(int(df['bkb'][i])) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token})
r.status_code

In [43]:
request1="""
match (u:UserNode) where u.state='Работа'
return u.name as user, u.username as Username, u.bkb_id as User_ID



"""
users=graph.run(request1).to_data_frame()

#Только русские пользаки
import re
rus_users=users[~users['user'].str.contains('[A-Za-z]+', flags=re.IGNORECASE, regex=True, na=False)]

rus_users.to_excel('rus_users.xlsx')

#print(list(rus_users['User_ID']))

#QC 2.0
request = """match (t:TaskNode)--(team:TeamNode {is_deleted:false}) 
where team.name contains 'Отдел обеспечения качества 2.0' and t.created_at contains '2019'
optional match (w:WitNode)--(tt:TaskTemplateNode)-[:BASE_FOR]-(t)
optional match (t)-[:CLIENT]-(c:UserNode)
return t.bkb_id as ID,t.name as Name, t.description as Description,tt.name as TaskTemplate,
t.user_name as User, t.customer_lead_time/3600 as CLT_hours, 
CASE
when exists (t.actual_start) then (t.actual_start_epoch-t.created_at_epoch)/3600
else (timestamp()/1000-t.created_at_epoch)/3600
END as BL_time_hours, 
CASE when exists (t.finish_time_epoch) then (t.finish_time_epoch-t.created_at_epoch)/3600
else 'не завершено' END as TLT_hours, c.erp_team as Client"""
df=graph.run(request).to_data_frame()
df=df[['ID','Name','Description','TaskTemplate','User','CLT_hours','BL_time_hours','TLT_hours','Client']]
df.to_excel('QC2.xlsx')

df.head(50)

In [3]:
#Поиск задач
df = graph.run("""
match (a:TaskNode)-[:CREATED_BY]-(b:UserNode) where b.name contains "BIMS" and a.external_url is null 
return a.bkb_id as task,a.name as name, a.status as status
""").to_data_frame()

print (df.head(10),'\n',df.shape)

                                                name   status    task
0                Пробоподготовка по MW деглико LC-HC  DISCARD  900918
1                Анализ образцов по MW деглико LC-HC  DISCARD  900917
2  Проведение обработки данных по анализу MW дегл...  DISCARD  900916
3  Пробоподготовка по MW глико / MW цельной молекулы  DISCARD  900674
4  Анализ образцов по MW глико / MW цельной молекулы  DISCARD  900673
5  Проведение обработки данных по анализу MW глик...  DISCARD  900672
6                            Подготовка хроматографа     PLAN  900544
7                                    Подготовка проб     PLAN  900543
8                                      Анализ данных     PLAN  900542
9                            Подготовка хроматографа     PLAN  900540 
 (2535, 3)


df=oom

r.text

#Поиск задач
df = graph.run("""
match (a:TaskNode)--(b:ProjectNode) where b.code in ["DP-82"]
return a.bkb_id as task,a.name as name, a.status as status
""").to_data_frame()

print (df.head(10),'\n',df.shape)

#Добавление наблюдателя
watcher_id = 5602
watchers = {}
for i in tqdm(range(df.shape[0])):
    watchers['task']=str(int(df['task'][i]))
    watchers['user']=watcher_id
    r = requests.post('https://repka.biocad.ru/api/v1/task-watchers/',
    json=watchers, headers = {'Authorization': 'Token ' + login_token})
    


#Ссылки для теста
i=0
for task in range(df.shape[0]):
    r = 'https://repka.biocad.ru/api/v1/tasks/' + str(df['task'][i]) + '/'
    i+=1
    print (r)

# Создание задач на пользователей из xlsx

In [191]:
#Подгрузка логинов из xlxs
df1=pd.read_excel('Сотрудники без плана развития.xlsx')
x=''
for n in range(df.shape[0]):
    x+='"'
    x+=df.Name[n]
    x+='",'
    
request = 'with ['+x[0:-1]+'] as names match (a:UserNode) where a.name in names return a.bkb_id as bkb_id'
df2 = graph.run(request).to_data_frame()
df2[0:5]


,bkb_id
0,5445
1,5349
2,5036
3,4311
4,4764


In [201]:
#Создание задач из xlsx
replaces = {}

#Дергаем API
for user in range(df2.shape[0]):
    replaces['name']='TaskName'
    replaces['description']='TaskDescription'
    replaces['deadline']='2019-06-01'
    replaces['client_username']='amosov'
    replaces['task_template']=11831 #Я не могу выбрать
    replaces['assigned_to']=df2['bkb_id'][user]
    replaces['priority']=400
    replaces['status']='NEW'
    #replaces['external_url']='https://bims-test.biocad.ru'
    #r = requests.post('https://repka.biocad.ru/api/v1/tasks/',
                       #json=replaces, headers = {'Authorization': 'Token ' + login_token})
    print(replaces)  
#r.status_code

{'name': 'TaskName', 'description': 'TaskDescription', 'deadline': '2019-06-01', 'client_username': 'amosov', 'task_template': 11831, 'assigned_to': 5445, 'priority': 400, 'status': 'NEW', 'external_url': 'https://bims-test.biocad.ru'}
{'name': 'TaskName', 'description': 'TaskDescription', 'deadline': '2019-06-01', 'client_username': 'amosov', 'task_template': 11831, 'assigned_to': 5349, 'priority': 400, 'status': 'NEW', 'external_url': 'https://bims-test.biocad.ru'}
{'name': 'TaskName', 'description': 'TaskDescription', 'deadline': '2019-06-01', 'client_username': 'amosov', 'task_template': 11831, 'assigned_to': 5036, 'priority': 400, 'status': 'NEW', 'external_url': 'https://bims-test.biocad.ru'}
{'name': 'TaskName', 'description': 'TaskDescription', 'deadline': '2019-06-01', 'client_username': 'amosov', 'task_template': 11831, 'assigned_to': 4311, 'priority': 400, 'status': 'NEW', 'external_url': 'https://bims-test.biocad.ru'}
{'name': 'TaskName', 'description': 'TaskDescription', '

In [8]:
bims_tasks=graph.run("""
match (a:TaskNode) where a.creator_name contains 'BIMS' and exists(a.external_url) 
and not a.status="READY" and not a.status='DISCARD' and not a.status='ARCHIVED'
return distinct a.bkb_id as id, a.name as name, a.task_template_name as TT, 
a.team_name as Team, a.user_name as User, a.status as Status
""").to_data_frame()
bims_tasks.to_excel('BIMS_Tasks.xlsx')

In [64]:
#Восстановление связей родителей из csv
tasks_restore=pd.read_csv('Tasks_Restore.csv')
replaces = {}

#Дергаем API
for i in tqdm(range(tasks_restore.shape[0])):
    replaces['parent']=str(tasks_restore['Parent'][i])
    r = requests.patch('https://repka.biocad.ru/api/v1/tasks/' + str(int(tasks_restore['Task'][i])) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token, 
                                                      'Content-Type': 'application/json'})
    print(str(i+1)+'/'+str(tasks_restore.shape[0]),' https://repka.biocad.ru/api/v1/tasks/' + str(int(tasks_restore['Task'][i])) + '/',str(replaces), \
          r.status_code)
 


0/358  https://repka.biocad.ru/api/v1/tasks/733825/ {'parent': '733824'} 200
1/358  https://repka.biocad.ru/api/v1/tasks/733826/ {'parent': '733824'} 200
2/358  https://repka.biocad.ru/api/v1/tasks/733827/ {'parent': '733824'} 200
3/358  https://repka.biocad.ru/api/v1/tasks/693370/ {'parent': '691677'} 200
4/358  https://repka.biocad.ru/api/v1/tasks/698485/ {'parent': '691677'} 200
5/358  https://repka.biocad.ru/api/v1/tasks/695012/ {'parent': '691677'} 200
6/358  https://repka.biocad.ru/api/v1/tasks/693391/ {'parent': '691677'} 200
7/358  https://repka.biocad.ru/api/v1/tasks/692147/ {'parent': '691677'} 200
8/358  https://repka.biocad.ru/api/v1/tasks/692167/ {'parent': '691677'} 200
9/358  https://repka.biocad.ru/api/v1/tasks/692117/ {'parent': '691677'} 200
10/358  https://repka.biocad.ru/api/v1/tasks/692096/ {'parent': '691677'} 200
11/358  https://repka.biocad.ru/api/v1/tasks/654346/ {'parent': '654343'} 200
12/358  https://repka.biocad.ru/api/v1/tasks/501789/ {'parent': '498966'} 

106/358  https://repka.biocad.ru/api/v1/tasks/749190/ {'parent': '746859'} 200
107/358  https://repka.biocad.ru/api/v1/tasks/741928/ {'parent': '738700'} 200
108/358  https://repka.biocad.ru/api/v1/tasks/737063/ {'parent': '734647'} 200
109/358  https://repka.biocad.ru/api/v1/tasks/737048/ {'parent': '734647'} 200
110/358  https://repka.biocad.ru/api/v1/tasks/742154/ {'parent': '741295'} 200
111/358  https://repka.biocad.ru/api/v1/tasks/741830/ {'parent': '736772'} 200
112/358  https://repka.biocad.ru/api/v1/tasks/743077/ {'parent': '742850'} 200
113/358  https://repka.biocad.ru/api/v1/tasks/737053/ {'parent': '734647'} 200
114/358  https://repka.biocad.ru/api/v1/tasks/743065/ {'parent': '742850'} 200
115/358  https://repka.biocad.ru/api/v1/tasks/741935/ {'parent': '738700'} 200
116/358  https://repka.biocad.ru/api/v1/tasks/749194/ {'parent': '746859'} 200
117/358  https://repka.biocad.ru/api/v1/tasks/742029/ {'parent': '738712'} 200
118/358  https://repka.biocad.ru/api/v1/tasks/763908

210/358  https://repka.biocad.ru/api/v1/tasks/706610/ {'parent': '704418'} 200
211/358  https://repka.biocad.ru/api/v1/tasks/756053/ {'parent': '755158'} 200
212/358  https://repka.biocad.ru/api/v1/tasks/707839/ {'parent': '707639'} 200
213/358  https://repka.biocad.ru/api/v1/tasks/741833/ {'parent': '736772'} 200
214/358  https://repka.biocad.ru/api/v1/tasks/742081/ {'parent': '738712'} 200
215/358  https://repka.biocad.ru/api/v1/tasks/706351/ {'parent': '704413'} 200
216/358  https://repka.biocad.ru/api/v1/tasks/707813/ {'parent': '707639'} 200
217/358  https://repka.biocad.ru/api/v1/tasks/708797/ {'parent': '708592'} 200
218/358  https://repka.biocad.ru/api/v1/tasks/708793/ {'parent': '708592'} 200
219/358  https://repka.biocad.ru/api/v1/tasks/742050/ {'parent': '738712'} 200
220/358  https://repka.biocad.ru/api/v1/tasks/708795/ {'parent': '708592'} 200
221/358  https://repka.biocad.ru/api/v1/tasks/707814/ {'parent': '707639'} 200
222/358  https://repka.biocad.ru/api/v1/tasks/706602

314/358  https://repka.biocad.ru/api/v1/tasks/690067/ {'parent': '687127'} 200
315/358  https://repka.biocad.ru/api/v1/tasks/674346/ {'parent': '673727'} 200
316/358  https://repka.biocad.ru/api/v1/tasks/689818/ {'parent': '687107'} 200
317/358  https://repka.biocad.ru/api/v1/tasks/689794/ {'parent': '687107'} 200
318/358  https://repka.biocad.ru/api/v1/tasks/689736/ {'parent': '687107'} 200
319/358  https://repka.biocad.ru/api/v1/tasks/690110/ {'parent': '687127'} 200
320/358  https://repka.biocad.ru/api/v1/tasks/673419/ {'parent': '672146'} 200
321/358  https://repka.biocad.ru/api/v1/tasks/746059/ {'parent': '745204'} 200
322/358  https://repka.biocad.ru/api/v1/tasks/674309/ {'parent': '672024'} 200
323/358  https://repka.biocad.ru/api/v1/tasks/662003/ {'parent': '661761'} 200
324/358  https://repka.biocad.ru/api/v1/tasks/689461/ {'parent': '689292'} 200
325/358  https://repka.biocad.ru/api/v1/tasks/689482/ {'parent': '689292'} 200
326/358  https://repka.biocad.ru/api/v1/tasks/667839

In [60]:

replaces = {}
replaces['parent']=''
r = requests.patch('https://repka.biocad.ru/api/v1/tasks/' + str(681327) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token, 
                                                      'Content-Type': 'application/json'})
print('https://repka.biocad.ru/api/v1/tasks/' + str(int(tasks_restore['Task'][i])) + '/',str(replaces), \
          r.status_code)

https://repka.biocad.ru/api/v1/tasks/669042/ {'parent': ''} 200


In [59]:

replaces = {}
replaces['status']="IN_PROGRESS"
r = requests.patch('https://repka.biocad.ru/api/v1/tasks/' + str(681327) + '/',
                       json=replaces, headers = {'Authorization': 'Token ' + login_token})
print('https://repka.biocad.ru/api/v1/tasks/' + str(int(tasks_restore['Task'][i])) + '/',str(replaces), \
          r.status_code)

https://repka.biocad.ru/api/v1/tasks/669042/ {'status': 'IN_PROGRESS'} 200
